# ``cpe_name`` Ingestion

Before CVEs can be ingested and processed for risk scoring, ``cpe_name``s must be identified and ingested as this is a required parameter when using the NVD CVE API. When no ``cpe_name`` is defined in the NVD CVE API call, all ``cpe_names`` are returned. 

## Scope Management

For the purposes of managing the scope of this application, a ``cpe_name`` selection process was developed using the following characteristics:

Target users/businesses:
* small to mid-sized public accounting firms
* mid to large-sized corporate internal accounting departments

Five (5) broad product categories:
* Accounting, Tax & Compliance
* Content & Analytics
* Core Business Systems
* Infrastructure & Platform
* Productivity & Collaboration

CPEs limited to part codes:
* ``:o:`` operating system
* ``:a:`` application software
* ``:h:`` hardware _(excluded)_

## Intended Purpose of Code

The below code was generated by AI to create a simple to use tool that interacts with the NVD CPE API.

Key features:
* User input to keyword search the CPE API
* Machine output results
* Saves search results to "../data/smb_software_to_cpe.csv"
    * Does not overwrite previous outputs to file
    * Appends new outputs to existing outputs in file

In [9]:
import csv, json, os, time, urllib.parse, requests, re
from pathlib import Path

# --- config -----------------------------------------------------------
API_URL   = "https://services.nvd.nist.gov/rest/json/cpes/2.0"
API_KEY = "ea5501a5-24fe-4720-80e3-2abed401d92f"
RATE_SECS = 1.2                               # stay well under the 5-sec/req soft limit

OUTPUT_CSV = Path("../data/smb_software_to_cpe.csv")
HEADER = ['CPE Name', 'Title']
# ----------------------------------------------------------------------

# --- function to search the CPE API -----------------------------------
def search_cpe_names(keyword):
    all_results = []
    start_index = 0
    results_per_page = 100  # Increase page size (max is 1000)
    
    headers = {
        "apiKey": API_KEY
    }

    while True:
        params = {
            "keywordSearch": keyword,
            "resultsPerPage": results_per_page,
            "startIndex": start_index
        }
        
        response = requests.get(API_URL, params=params, headers=headers)
        
        if response.status_code != 200:
            print(f"Error fetching data for '{keyword}': {response.status_code} - {response.text}")
            break
        
        data = response.json()
        cpe_matches = data.get('products', [])
        if not cpe_matches:
            break
        
        for item in cpe_matches:
            cpe_uri = item.get('cpe', {}).get('cpeName', '')
            metadata = item.get('cpe', {}).get('titles', [])
            title = next((t['title'] for t in metadata if t.get('lang') == 'en'), metadata[0]['title'] if metadata else '')
            if cpe_uri:
                all_results.append({'cpeName': cpe_uri, 'title': title})
        
        total_results = data.get('totalResults', 0)
        start_index += results_per_page
        
        if start_index >= total_results:
            break
        
        time.sleep(RATE_SECS)
    
    return all_results

# --- main interactive loop ---------------------------------------------
while True:
    search_input = input("Enter product search keywords separated by commas (or type 'exit' to quit): ").strip()
    if search_input.lower() == "exit":
        print("Exiting program. Goodbye!")
        break

    search_keywords = [kw.strip() for kw in search_input.split(',')]
    print("Search Keywords:", search_keywords)

    all_results = []
    for keyword in search_keywords:
        if keyword.lower() == "exit":
            print("Exiting program. Goodbye!")
            break
        time.sleep(RATE_SECS)
        matches = search_cpe_names(keyword)
        print(f"\nMatches for '{keyword}':")
        print(f"Number of matching results: {len(matches)}")
        if matches:
            for match in matches:
                print(f"   Title: {match['title']}")
                print(f" - CPE Name: {match['cpeName']}")
        else:
            print("No matches found.")
        all_results.extend(matches)

    if all_results:
        while True:
            save_input = input(f"\nDo you want to save {len(matches)} results to file? (yes/no/exit): ").strip().lower()
            if save_input == "yes" or save_input == "y":
                write_header = True
                if OUTPUT_CSV.exists():
                    with open(OUTPUT_CSV, "r", encoding='utf-8') as csvfile:
                        reader = csv.reader(csvfile)
                        first_row = next(reader, None)
                        if first_row and all(header in first_row for header in HEADER):
                            write_header = False

                with open(OUTPUT_CSV, "a", newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    if write_header:
                        writer.writerow(HEADER)
                    for result in all_results:
                        writer.writerow([result['cpeName'], result['title']])
                print(f"\n{len(matches)} results appended to {OUTPUT_CSV.resolve()}")
                break  # exit the save prompt loop
            elif save_input == "no" or save_input == "n":
                print("\nYou chose not to save. Please enter new keywords.")
                break  # exit the save prompt loop
            elif save_input == "exit" or save_input == "e":
                print("Exiting program. Goodbye!")
                break
            else:
                print("Invalid input. Please enter 'yes', 'no', or 'exit'.")
    else:
        print("\nNo results to save. Please enter new keywords.")

Enter product search keywords separated by commas (or type 'exit' to quit):  vmware workstation


Search Keywords: ['vmware workstation']

Matches for 'vmware workstation':
Number of matching results: 200
   Title: VMWare VMWare
 - CPE Name: cpe:2.3:a:vmware:workstation:-:*:*:*:*:*:*:*
   Title: VMWare VMWare 2.0
 - CPE Name: cpe:2.3:a:vmware:workstation:2.0:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 3.2.1 patch1
 - CPE Name: cpe:2.3:a:vmware:workstation:3.2.1:patch1:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 3.4
 - CPE Name: cpe:2.3:a:vmware:workstation:3.4:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 4.0
 - CPE Name: cpe:2.3:a:vmware:workstation:4.0:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 4.0.1
 - CPE Name: cpe:2.3:a:vmware:workstation:4.0.1:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 4.0.1 build5289
 - CPE Name: cpe:2.3:a:vmware:workstation:4.0.1_build_5289:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 4.0.2
 - CPE Name: cpe:2.3:a:vmware:workstation:4.0.2:*:*:*:*:*:*:*
   Title: VMWare VMWare Workstation 4.5.2
 - CPE Name: cpe:2.3:a:vmware:wo


Do you want to save 200 results to file? (yes/no/exit):  y



200 results appended to C:\Users\hgbtx\Desktop\MIS433\final-project\cyber-risk-scoring\data\smb_software_to_cpe.csv


Enter product search keywords separated by commas (or type 'exit' to quit):  vmware vsphere


...


Do you want to save 95 results to file? (yes/no/exit):  y



95 results appended to C:\Users\hgbtx\Desktop\MIS433\final-project\cyber-risk-scoring\data\smb_software_to_cpe.csv


Enter product search keywords separated by commas (or type 'exit' to quit):  e


Search Keywords: ['e']


KeyboardInterrupt: 